In [1]:
setwd("/Users/pranavkulkarni/SDM/Climate_Models_Arenaviruses/")
library(raster)
library(terra)
library(tidyverse)

Loading required package: sp

terra 1.7.78

── Attaching core tidyverse packages ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::extract() masks terra::extract(), raster::extract()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ dplyr::select()  masks raster::select()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
curr_bclims <- rast(list.files(path = "./Data/Input/Processed/Resampled/machu",
                     full.names =  T))

ssp2_bclims <- rast(list.files(path = "./Data/Input/Processed/Projected/machu/SSP2",
                     full.names =  T))

ssp5_bclims <- rast(list.files(path = "./Data/Input/Processed/Projected/machu/SSP5",
                     full.names =  T))

In [3]:
curr_foi <- rast("./Data/Output/iterations/C_callosus/current/foi.tif")
ssp2_foi <- rast("./Data/Output/iterations/C_callosus/ssp2/foi_ssp2.tif")
ssp5_foi <- rast("./Data/Output/iterations/C_callosus/ssp5/foi_ssp5.tif")

In [4]:
delta_ssp2_bclims <- ssp2_bclims - curr_bclims
delta_ssp5_bclims <- ssp5_bclims - curr_bclims


delta_ssp2_foi <- ssp2_foi - curr_foi
delta_ssp5_foi <- ssp5_foi - curr_foi

## SSP2

In [5]:
mat_delta_ssp2_foi <- as.matrix(values(delta_ssp2_foi))
mat_delta_ssp2_bioclims <- as.matrix(values(delta_ssp2_bclims))
delta_ssp2_mod_df <- as.data.frame(cbind(mat_delta_ssp2_foi, mat_delta_ssp2_bioclims))
delta_ssp2_mod_df <- delta_ssp2_mod_df %>% 
  select(-c(ndvi, dem)) %>% 
  mutate(foi = floor(gpw_v4_2000*1000)) %>% 
  select(-gpw_v4_2000) %>%
  drop_na(.)

rm(mat_delta_ssp2_bioclims,mat_delta_ssp2_foi)

In [6]:
if (!dir.exists(file.path("./Data/Output/Iterations/C_callosus/ssp2/foi_mod_plots"))) {dir.create("./Data/Output/Iterations/C_callosus/ssp2/foi_mod_plots")}

In [7]:
model_delta_ssp2 <- lm(foi ~., data = delta_ssp2_mod_df)

summary(model_delta_ssp2)



Call:
lm(formula = foi ~ ., data = delta_ssp2_mod_df)

Residuals:
    Min      1Q  Median      3Q     Max 
-34.908  -1.144  -0.053   0.824  30.746 

Coefficients:
                           Estimate Std. Error t value Pr(>|t|)    
(Intercept)              -6.412e+00  1.595e-01 -40.194  < 2e-16 ***
bclim_01                  2.647e+00  1.209e-01  21.896  < 2e-16 ***
bclim_02                  7.495e+00  1.274e-01  58.850  < 2e-16 ***
bclim_03                  1.230e-01  1.876e-02   6.555 5.56e-11 ***
bclim_04                 -2.876e-03  2.296e-03  -1.253 0.210245    
bclim_05                 -8.793e+03  5.599e+03  -1.570 0.116316    
bclim_06                  8.795e+03  5.599e+03   1.571 0.116240    
bclim_07                  8.793e+03  5.599e+03   1.570 0.116301    
bclim_08                  5.511e-01  2.936e-02  18.769  < 2e-16 ***
bclim_09                 -1.623e-01  1.686e-02  -9.624  < 2e-16 ***
bclim_10                  2.418e-01  8.557e-02   2.825 0.004722 ** 
bclim_11            

In [8]:
for (i in 1:24) {
  
  pl <- ggplot(delta_ssp2_mod_df, aes(x = delta_ssp2_mod_df[,i], y = delta_ssp2_mod_df[,25])) + 
    geom_point(alpha = 0.3) +
    labs(x = names(delta_ssp2_mod_df)[i], y = "FOI x 1000", subtitle = "red: lm, green: binomial, blue: gam") +
    stat_smooth(method = "lm", col = "red", se = T) +
    stat_smooth(method = "auto", col = "blue", se = T) +
    theme_classic()
  ggsave(filename = paste0("./Data/Output/Iterations/C_callosus/ssp2/foi_mod_plots/foi_mod_", i, ".png"), 
         plot = pl, device = "png",
         width = 11, height = 8.5, units = "in")
  
}

`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ 

## SSP5

In [9]:
mat_delta_ssp5_foi <- as.matrix(values(delta_ssp5_foi))
mat_delta_ssp5_bioclims <- as.matrix(values(delta_ssp5_bclims))
delta_ssp5_mod_df <- as.data.frame(cbind(mat_delta_ssp5_foi, mat_delta_ssp5_bioclims))
delta_ssp5_mod_df <- delta_ssp5_mod_df %>% 
  select(-c(ndvi, dem)) %>% 
  mutate(foi = floor(gpw_v4_2000*1000)) %>% 
  select(-gpw_v4_2000) %>%
  drop_na(.)

rm(mat_delta_ssp5_bioclims,mat_delta_ssp5_foi)

In [10]:
if (!dir.exists(file.path("./Data/Output/Iterations/C_callosus/ssp5/foi_mod_plots"))) {dir.create("./Data/Output/Iterations/C_callosus/ssp5/foi_mod_plots")}

In [11]:
model_delta_ssp5 <- lm(foi ~., data = delta_ssp5_mod_df)

summary(model_delta_ssp5)



Call:
lm(formula = foi ~ ., data = delta_ssp5_mod_df)

Residuals:
    Min      1Q  Median      3Q     Max 
-29.027  -1.177  -0.056   0.832  32.697 

Coefficients:
                           Estimate Std. Error t value Pr(>|t|)    
(Intercept)              -6.863e+00  1.769e-01 -38.808  < 2e-16 ***
bclim_01                  2.846e+00  1.025e-01  27.776  < 2e-16 ***
bclim_02                  6.447e+00  1.041e-01  61.940  < 2e-16 ***
bclim_03                  4.651e-02  1.700e-02   2.737  0.00621 ** 
bclim_04                 -1.556e-02  2.031e-03  -7.661 1.85e-14 ***
bclim_05                 -4.343e+03  5.582e+03  -0.778  0.43649    
bclim_06                  4.345e+03  5.582e+03   0.778  0.43635    
bclim_07                  4.343e+03  5.582e+03   0.778  0.43653    
bclim_08                  6.876e-01  2.766e-02  24.864  < 2e-16 ***
bclim_09                 -2.627e-01  1.595e-02 -16.471  < 2e-16 ***
bclim_10                  3.570e-01  7.970e-02   4.479 7.51e-06 ***
bclim_11            

In [12]:

for (i in 1:24) {
  
  pl <- ggplot(delta_ssp5_mod_df, aes(x = delta_ssp5_mod_df[,i], y = delta_ssp5_mod_df[,25])) + 
    geom_point(alpha = 0.3) +
    labs(x = names(delta_ssp5_mod_df)[i], y = "FOI x 1000", subtitle = "red: lm, green: binomial, blue: gam") +
    stat_smooth(method = "lm", col = "red", se = T) +
    stat_smooth(method = "auto", col = "blue", se = T) +
    theme_classic()
  ggsave(filename = paste0("./Data/Output/Iterations/C_callosus/ssp5/foi_mod_plots/foi_mod_", i, ".png"), 
         plot = pl, device = "png",
         width = 11, height = 8.5, units = "in")
  
}

`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ 